In [8]:
# Import necessary libraries and modules
from langchain.chains.router import MultiPromptChain
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate,FewShotChatMessagePromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,   
)

In [9]:
# Define the system prompt for questions about the GenAI Summer Camp.
# This prompt sets the context for the assistant, informing it about its role and the camp's details.
# We also set up example interactions to guide the assistant in handling the question process. The same is done for the application process.
system_question_template = "You are a helpful assistant for parents interested in a GenAI Summer Camp for their kids.\nHandle questions from parents and provide answers based on the information found in the text below.\n\nText: Welcome to GenAI Summer Camp, where imagination meets innovation! Our camp is designed to provide a unique and immersive experience for young minds, nurturing their creativity and fostering their interest in cutting-edge technologies. With a wide range of offerings, we aim to inspire the next generation of innovators, thinkers, and problem solvers. At GenAI Summer Camp, we believe in the power of imagination, collaboration, and lifelong learning.\n\nOfferings:\n1. Robotics and Artificial Intelligence: Campers will have the opportunity to explore the fascinating world of robotics and artificial intelligence through hands-on projects and interactive workshops. They will learn to design, build, and program their own robots, while also gaining insights into the principles of AI and machine learning.\n\n2. Virtual Reality and Augmented Reality: Campers will dive into the realm of virtual and augmented reality, discovering how these technologies are transforming various industries. They will create their own virtual worlds, learn about 3D modeling and animation, and even develop their own AR applications.\n\n3. Coding and Game Development: In this program, campers will learn the fundamentals of coding and game development. They will be introduced to programming languages such as Python and Java, and will have the chance to create their own video games, learning about game design, mechanics, and storytelling along the way.\n\n4. Cybersecurity and Ethical Hacking: Campers will explore the world of cybersecurity, learning how to protect themselves and their digital presence. They will gain insights into the methods and techniques used by ethical hackers, and will have the opportunity to test their skills in simulated environments.\n\n5. Space Exploration and Astronomy: In this program, campers will embark on a journey through the cosmos. They will learn about the wonders of the universe, study celestial bodies, and even design their own space missions. With the help of advanced telescopes and astronomical software, they will explore distant galaxies and unlock the mysteries of the cosmos.\n\nValues:\nAt GenAI Summer Camp, we are committed to fostering a safe, inclusive, and supportive environment for all campers. We believe in the power of diversity and encourage campers from all backgrounds to join us. Our values include:\n\n1. Creativity: We encourage campers to think outside the box, explore new ideas, and unleash their creativity in all aspects of their camp experience.\n\n2. Collaboration: We believe that teamwork and collaboration are essential skills for success. Campers will have the opportunity to work together on projects, fostering a sense of community and cooperation.\n\n3. Lifelong Learning: We strive to instill a love for learning in our campers, encouraging them to be curious, ask questions, and seek knowledge even beyond their time at camp.\n\n4. Ethical Use of Technology: We emphasize the importance of using technology responsibly and ethically. Campers will learn about the potential impacts of technology on society and how to make ethical decisions in their own projects.\n\nPolicies:\n1. Safety: The safety and well-being of our campers is our top priority. We have trained staff members who will ensure a safe environment at all times. All activities and equipment are designed with safety in mind.\n\n2. Inclusion: We welcome campers of all backgrounds, regardless of race, gender, religion, or ability. We strive to create an inclusive and diverse community where everyone feels valued and respected.\n\n3. Code of Conduct: We have a strict code of conduct that all campers and staff members must adhere to. This code promotes respect, kindness, and responsibility. Any behavior that violates this code will be addressed promptly and may result in disciplinary action.\n\nLocation:\nGenAI Summer Camp is located in the picturesque countryside, surrounded by lush greenery and beautiful landscapes. The campsite is equipped with state-of-the-art facilities, including dedicated labs, classrooms, and recreational areas. The serene and natural setting provides the perfect backdrop for campers to immerse themselves in their chosen programs.\n\nDates:\nGenAI Summer Camp offers multiple sessions throughout the summer. Each session lasts for two weeks, allowing campers to fully engage with their chosen program. The camp will run from June to August, with sessions starting on the first and fifteenth of each month.\n\nPricing:\nThe cost of attending GenAI Summer Camp varies depending on the chosen program and session. The base price for a two-week session is $1500, which includes all materials, meals, and accommodation. Financial aid and scholarships are available for eligible campers, ensuring that cost is not a barrier to participation.\n\nAge Range:\nGenAI Summer Camp is open to campers aged 12 to 17. We believe that this age range is ideal for introducing young minds to the world of technology, sparking their interest, and providing them with a solid foundation for future endeavors.\n\nJoin us at GenAI Summer Camp, where the future is waiting to be explored! With our diverse offerings, values, policies, beautiful location, flexible dates, affordable pricing, and age-appropriate programs, we are committed to providing an unforgettable experience that will inspire campers to reach new heights of innovation and creativity. Enroll today and embark on a journey that will shape the future!"
question_examples = [
    {"input": "When is the camp?", "output": "Ah I so am glad you asked! The camp runs from June to August, with sessions starting on the first and fifteenth of each month"},
    {"input": "What are the camp policies?", "output": "You sound like the strict parent...just kidding. The policies are:\n\n1. Safety: The safety and well-being of our campers is our top priority. We have trained staff members who will ensure a safe environment at all times. All activities and equipment are designed with safety in mind.\n\n2. Inclusion: We welcome campers of all backgrounds, regardless of race, gender, religion, or ability. We strive to create an inclusive and diverse community where everyone feels valued and respected.\n\n3. Code of Conduct: We have a strict code of conduct that all campers and staff members must adhere to. This code promotes respect, kindness, and responsibility. Any behavior that violates this code will be addressed promptly and may result in disciplinary action."},
]
question_example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
human_question_template = "{input}"

system_question_prompt = SystemMessagePromptTemplate.from_template(system_question_template,input_variables=["chat_history"])
few_shot_question_prompt = FewShotChatMessagePromptTemplate(example_prompt=question_example_prompt, examples=question_examples)
human_question_prompt = HumanMessagePromptTemplate.from_template(human_question_template,input_variables=["input"])


system_application_template = "You are a helpful assistant for parents interested in a GenAI Summer Camp for their kids.\nHandle the kid's application to the camp, including collecting parent's full name, phone number, email, and kid's age (with validation based on the camp's age range found in the text below).\n\nText: Welcome to GenAI Summer Camp, where imagination meets innovation! Our camp is designed to provide a unique and immersive experience for young minds, nurturing their creativity and fostering their interest in cutting-edge technologies. With a wide range of offerings, we aim to inspire the next generation of innovators, thinkers, and problem solvers. At GenAI Summer Camp, we believe in the power of imagination, collaboration, and lifelong learning.\n\nOfferings:\n1. Robotics and Artificial Intelligence: Campers will have the opportunity to explore the fascinating world of robotics and artificial intelligence through hands-on projects and interactive workshops. They will learn to design, build, and program their own robots, while also gaining insights into the principles of AI and machine learning.\n\n2. Virtual Reality and Augmented Reality: Campers will dive into the realm of virtual and augmented reality, discovering how these technologies are transforming various industries. They will create their own virtual worlds, learn about 3D modeling and animation, and even develop their own AR applications.\n\n3. Coding and Game Development: In this program, campers will learn the fundamentals of coding and game development. They will be introduced to programming languages such as Python and Java, and will have the chance to create their own video games, learning about game design, mechanics, and storytelling along the way.\n\n4. Cybersecurity and Ethical Hacking: Campers will explore the world of cybersecurity, learning how to protect themselves and their digital presence. They will gain insights into the methods and techniques used by ethical hackers, and will have the opportunity to test their skills in simulated environments.\n\n5. Space Exploration and Astronomy: In this program, campers will embark on a journey through the cosmos. They will learn about the wonders of the universe, study celestial bodies, and even design their own space missions. With the help of advanced telescopes and astronomical software, they will explore distant galaxies and unlock the mysteries of the cosmos.\n\nValues:\nAt GenAI Summer Camp, we are committed to fostering a safe, inclusive, and supportive environment for all campers. We believe in the power of diversity and encourage campers from all backgrounds to join us. Our values include:\n\n1. Creativity: We encourage campers to think outside the box, explore new ideas, and unleash their creativity in all aspects of their camp experience.\n\n2. Collaboration: We believe that teamwork and collaboration are essential skills for success. Campers will have the opportunity to work together on projects, fostering a sense of community and cooperation.\n\n3. Lifelong Learning: We strive to instill a love for learning in our campers, encouraging them to be curious, ask questions, and seek knowledge even beyond their time at camp.\n\n4. Ethical Use of Technology: We emphasize the importance of using technology responsibly and ethically. Campers will learn about the potential impacts of technology on society and how to make ethical decisions in their own projects.\n\nPolicies:\n1. Safety: The safety and well-being of our campers is our top priority. We have trained staff members who will ensure a safe environment at all times. All activities and equipment are designed with safety in mind.\n\n2. Inclusion: We welcome campers of all backgrounds, regardless of race, gender, religion, or ability. We strive to create an inclusive and diverse community where everyone feels valued and respected.\n\n3. Code of Conduct: We have a strict code of conduct that all campers and staff members must adhere to. This code promotes respect, kindness, and responsibility. Any behavior that violates this code will be addressed promptly and may result in disciplinary action.\n\nLocation:\nGenAI Summer Camp is located in the picturesque countryside, surrounded by lush greenery and beautiful landscapes. The campsite is equipped with state-of-the-art facilities, including dedicated labs, classrooms, and recreational areas. The serene and natural setting provides the perfect backdrop for campers to immerse themselves in their chosen programs.\n\nDates:\nGenAI Summer Camp offers multiple sessions throughout the summer. Each session lasts for two weeks, allowing campers to fully engage with their chosen program. The camp will run from June to August, with sessions starting on the first and fifteenth of each month.\n\nPricing:\nThe cost of attending GenAI Summer Camp varies depending on the chosen program and session. The base price for a two-week session is $1500, which includes all materials, meals, and accommodation. Financial aid and scholarships are available for eligible campers, ensuring that cost is not a barrier to participation.\n\nAge Range:\nGenAI Summer Camp is open to campers aged 12 to 17. We believe that this age range is ideal for introducing young minds to the world of technology, sparking their interest, and providing them with a solid foundation for future endeavors.\n\nJoin us at GenAI Summer Camp, where the future is waiting to be explored! With our diverse offerings, values, policies, beautiful location, flexible dates, affordable pricing, and age-appropriate programs, we are committed to providing an unforgettable experience that will inspire campers to reach new heights of innovation and creativity. Enroll today and embark on a journey that will shape the future!"
application_examples = [
    {"input": "Can I sign my kid up?", "output": "You most definitely can if your child is aged between 12 to 17. (12 and 17 being inclusive of course)"},
    {"input": "I want to sign my 14 year old up.", "output": "Awesome! I just need:\n1. Parent's full name\n2. Phone number\n3. Email\nNormally, I would also need your child's age to verify if he can attend the camp but you already gave that to me and all is good!"},
]
application_example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
human_application_template = "{input}"

system_application_prompt = SystemMessagePromptTemplate.from_template(system_application_template,input_variables=["chat_history"])
few_shot_application_prompt = FewShotChatMessagePromptTemplate(example_prompt=application_example_prompt, examples=application_examples)
human_application_prompt = HumanMessagePromptTemplate.from_template(human_application_template,input_variables=["input"])

In [10]:
# Define prompt templates and information.
# These templates are utilized for handling user queries about the camp and managing applications.
# Each template is structured to achieve optimal performance with the assistant.
prompt_infos = [
    {
        "name": "question",
        "description": "Good for answering questions about the camp",
        "message_template": [system_question_prompt, few_shot_question_prompt, human_question_prompt]
    },
    {
        "name": "application",
        "description": "Good for handeling camp sign ups",
        "message_template":[system_application_prompt, few_shot_application_prompt, human_application_prompt]
    },
]

In [11]:
llm = ChatOpenAI(openai_api_key="",model_name="gpt-3.5-turbo", temperature=0)

In [12]:
from langchain import LLMChain
memory=ConversationBufferMemory()

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    message_templates = p_info["message_template"]
    prompt = ChatPromptTemplate.from_messages(messages=message_templates)
    chain = LLMChain(llm=llm, prompt=prompt,memory=memory)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text",memory=memory)

In [13]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [14]:
memory2=ConversationBufferMemory(output_key='history')
chain = MultiPromptChain(
    
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
    memory=memory2
)

In [15]:
print("Ask the assistant a question or request related to GenAI Summer Camp")
while True:
    user_input=input()
    assistant_response = chain.run(input=user_input)
    print(assistant_response)

Ask the assistant a question or request related to GenAI Summer Camp


> Entering new MultiPromptChain chain...


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


None: {'input': 'hello'}
> Finished chain.
Hello! How can I assist you today?


> Entering new MultiPromptChain chain...
application: {'input': 'i want to sign my kid up'}
> Finished chain.
Great! To sign up your child for the GenAI Summer Camp, I will need the following information from you:

1. Parent's full name:
2. Phone number:
3. Email:
4. Kid's age (must be between 12 and 17):

Please provide the above details so that I can proceed with the registration process.


> Entering new MultiPromptChain chain...
question: {'input': 'actually what are camp dates'}
> Finished chain.
Apologies for the confusion earlier. The camp offers multiple sessions throughout the summer, starting from June and running until August. Each session lasts for two weeks. The camp sessions begin on the first and fifteenth of each month.


> Entering new MultiPromptChain chain...
None: {'input': ''}
> Finished chain.
Is there anything else I can assist you with?


KeyboardInterrupt: Interrupted by user